In [1]:
!git clone https://github.com/brimmann/llm-recipes-2.git

Cloning into 'llm-recipes-2'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.6)
remote: Compressing objects: 100% (85/85), done.
remote: Total 116 (delta 42), reused 102 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (116/116), 311.20 KiB | 712.00 KiB/s, done.
Resolving deltas: 100% (42/42), done.


In [ ]:
%cd /content/llm-recipes-2

In [2]:
from train.train_utils import train
from configs.configs_utils import update_config
from data.data_utils import (get_dataloader, get_distillation_dataloader)
from train.tools import (setup, setup_environ_flags, clear_gpu_cache)
from models.models_utils import  get_optimizer
from models.models_utils import get_distillation_models
from data.data_utils import get_dataloader
import torch

/home/brimmann/works/llm-recipes-2/.venv/lib/python3.9/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/brimmann/works/llm-recipes-2/.venv/lib/python3.9/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have ha

In [3]:
from configs.datasets import dataset as DatasetConfig
from configs.training import train_config as TrainConfig
from configs.distillation import distillation_config as DistillationConfig
from configs.fsdp import fsdp_config as FsdpConfig

dataset_config = DatasetConfig()
train_config = TrainConfig()
distill_config = DistillationConfig()
fsdp_config = DistillationConfig()
train_config.model_name = "google/gemma-3-1b-it"
distill_config.model_name = "Hannibal046/xrag-7b"
train_config.batch_size_training = 8
train_config.distillation = True
train_config.num_workers_dataloader = 0
train_config.num_epochs = 5
dataset_config.file = "data/loaders/squad-v2-sampled.py"

In [4]:
rank = 0

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
student_tokenizer, teacher_tokenizer, model = get_distillation_models(train_config, distill_config, fsdp_config, rank)

--> Model google/gemma-3-1b-it

--> google/gemma-3-1b-it has 999.885952 Million params



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
teacher_train_dataloader, teacher_eval_dataloader = get_dataloader(dataset_config, train_config, teacher_tokenizer, rank, distill_config)
student_train_dataloader, student_eval_dataloader = get_dataloader(dataset_config, train_config, student_tokenizer, rank, distill_config)

In [ ]:
optimizer = get_optimizer(model, train_config, fsdp_config)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=train_config.lr, epochs=train_config.num_epochs, steps_per_epoch=len(student_train_dataloader),
                                                pct_start=train_config.pct_start, div_factor=train_config.div_factor, final_div_factor=train_config.final_div_factor)

In [ ]:
results = train(
      model,
      student_train_dataloader,
      student_eval_dataloader,
      optimizer,
      scheduler,
      train_config.gradient_accumulation_steps,
      train_config,
      distill_config,
      dataset_config,
      teacher_train_dataloader if train_config.distillation else None,
      teacher_eval_dataloader if train_config.distillation else None,
      fsdp_config if train_config.enable_fsdp else None,
      None,
      rank,
  )
if rank == 0:
    [print(f'Key: {k}, Value: {v}') for k, v in results.items()]